# Python网络安全练习 - 网络安全工具

本练习涵盖网络扫描、端口检测、密码安全等实用工具的开发。

**重要提示**: 这些工具仅用于教育目的和授权的安全测试。未经授权扫描他人网络是违法的！

## 练习1: 端口扫描器

**任务**: 编写一个简单的端口扫描器

**知识点**: TCP连接、Socket编程

**应用**: 检测目标主机上开放的端口

In [ ]:
import socket
from datetime import datetime

def scan_port(host, port, timeout=1):
    """
    扫描单个端口
    
    参数:
        host: 目标主机
        port: 端口号
        timeout: 超时时间(秒)
    
    返回:
        True 如果端口开放，否则 False
    """
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(timeout)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except socket.error:
        return False

# 示例: 扫描本地常见端口
host = "127.0.0.1"
common_ports = [21, 22, 23, 25, 80, 443, 3306, 5432, 8080]

print(f"开始扫描主机: {host}")
print(f"时间: {datetime.now()}\n")

for port in common_ports:
    if scan_port(host, port):
        print(f"端口 {port}: 开放")
    else:
        print(f"端口 {port}: 关闭")

print(f"\n扫描完成: {datetime.now()}")

In [ ]:
# 练习: 改进端口扫描器
# 添加多线程支持以加快扫描速度

import threading

def threaded_port_scanner(host, start_port, end_port, max_threads=50):
    """
    多线程端口扫描器
    
    参数:
        host: 目标主机
        start_port: 起始端口
        end_port: 结束端口
        max_threads: 最大线程数
    """
    open_ports = []
    lock = threading.Lock()
    
    def scan_port_thread(port):
        if scan_port(host, port, timeout=0.5):
            with lock:
                open_ports.append(port)
                print(f"发现开放端口: {port}")
    
    # TODO: 实现多线程扫描逻辑
    # 提示: 使用 threading.Thread 和 thread.join()
    
    return sorted(open_ports)

# 测试 (仅扫描本地)
# result = threaded_port_scanner("127.0.0.1", 1, 1024)
# print(f"\n开放的端口: {result}")

## 练习2: Banner Grabbing (服务识别)

**任务**: 获取开放端口的服务信息

**知识点**: 通过Banner识别服务类型和版本

**应用**: 识别目标系统运行的服务

In [1]:
import socket

def grab_banner(host, port, timeout=2):
    """
    获取服务Banner信息
    
    参数:
        host: 目标主机
        port: 端口号
        timeout: 超时时间
    
    返回:
        Banner字符串
    """
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(timeout)
        sock.connect((host, port))
        
        # 发送HTTP请求或等待服务器响应
        sock.send(b"HEAD / HTTP/1.0\r\n\r\n")
        banner = sock.recv(1024).decode('utf-8', errors='ignore').strip()
        
        sock.close()
        return banner
    except Exception as e:
        return f"错误: {str(e)}"

# 示例
# banner = grab_banner("example.com", 80)
# print(f"服务Banner:\n{banner}")

## 练习3: 子域名枚举

**任务**: 通过字典枚举子域名

**知识点**: DNS查询

**应用**: 发现目标域名的子域

In [ ]:
import socket

def enumerate_subdomains(domain, wordlist):
    """
    枚举子域名
    
    参数:
        domain: 主域名 (例如: example.com)
        wordlist: 子域名字典列表
    
    返回:
        发现的子域名列表
    """
    found_subdomains = []
    
    for subdomain in wordlist:
        full_domain = f"{subdomain}.{domain}"
        try:
            # 尝试解析域名
            ip = socket.gethostbyname(full_domain)
            found_subdomains.append((full_domain, ip))
            print(f"[+] 发现: {full_domain} -> {ip}")
        except socket.gaierror:
            # 域名不存在
            pass
    
    return found_subdomains

# 示例使用 (使用常见子域名)
common_subdomains = ['www', 'mail', 'ftp', 'admin', 'test', 'dev', 'api']

# 注意: 请使用你自己的域名进行测试
# results = enumerate_subdomains("example.com", common_subdomains)
# print(f"\n共发现 {len(results)} 个子域名")

## 练习4: 密码生成器

**任务**: 创建一个安全的随机密码生成器

**知识点**: 随机数生成、密码安全

In [ ]:
import secrets
import string

def generate_password(length=16, use_upper=True, use_lower=True, 
                      use_digits=True, use_special=True):
    """
    生成安全的随机密码
    
    参数:
        length: 密码长度
        use_upper: 是否包含大写字母
        use_lower: 是否包含小写字母
        use_digits: 是否包含数字
        use_special: 是否包含特殊字符
    
    返回:
        生成的密码
    """
    characters = ""
    
    if use_upper:
        characters += string.ascii_uppercase
    if use_lower:
        characters += string.ascii_lowercase
    if use_digits:
        characters += string.digits
    if use_special:
        characters += string.punctuation
    
    if not characters:
        raise ValueError("至少选择一种字符类型")
    
    # 使用 secrets 模块生成安全的随机密码
    password = ''.join(secrets.choice(characters) for _ in range(length))
    
    return password

# 示例: 生成多个密码
print("生成的安全密码:\n")
for i in range(5):
    pwd = generate_password(12)
    print(f"{i+1}. {pwd}")

In [ ]:
# 练习: 密码短语生成器
# 使用随机词组生成易记但安全的密码

import secrets

def generate_passphrase(word_count=4, separator='-'):
    """
    生成密码短语 (类似: correct-horse-battery-staple)
    
    参数:
        word_count: 单词数量
        separator: 分隔符
    
    返回:
        密码短语
    """
    # 示例词库 (实际应用中应使用更大的词库)
    words = [
        'apple', 'banana', 'cherry', 'dragon', 'eagle', 'falcon',
        'guitar', 'hammer', 'island', 'jungle', 'knight', 'lemon',
        'mountain', 'ninja', 'ocean', 'piano', 'queen', 'robot',
        'sunset', 'tiger', 'umbrella', 'violet', 'wizard', 'yellow'
    ]
    
    # TODO: 实现密码短语生成
    # 提示: 使用 secrets.choice() 随机选择单词
    pass

# 测试
# for i in range(3):
#     print(generate_passphrase())

## 练习5: 网络数据包嗅探 (基础)

**任务**: 使用 scapy 捕获和分析网络数据包

**知识点**: 网络协议、数据包分析

**注意**: 需要安装 scapy: `pip install scapy`

**权限**: 需要管理员/root权限

In [ ]:
# 取消注释安装 scapy
# !pip install scapy

from scapy.all import *

def packet_callback(packet):
    """
    数据包回调函数
    """
    if packet.haslayer(IP):
        ip_src = packet[IP].src
        ip_dst = packet[IP].dst
        protocol = packet[IP].proto
        
        print(f"IP包: {ip_src} -> {ip_dst} (协议: {protocol})")
        
        if packet.haslayer(TCP):
            print(f"  TCP: {packet[TCP].sport} -> {packet[TCP].dport}")
        elif packet.haslayer(UDP):
            print(f"  UDP: {packet[UDP].sport} -> {packet[UDP].dport}")

# 示例: 捕获数据包 (需要管理员权限)
# print("开始捕获数据包 (Ctrl+C 停止)...")
# sniff(prn=packet_callback, count=10)  # 捕获10个数据包

## 练习6: IP地理位置查询

**任务**: 查询IP地址的地理位置信息

**知识点**: API调用、JSON解析

In [1]:
import requests
import json

def get_ip_location(ip_address):
    """
    查询IP地址的地理位置
    
    参数:
        ip_address: IP地址
    
    返回:
        包含位置信息的字典
    """
    try:
        # 使用免费的IP地理位置API
        url = f"http://ip-api.com/json/{ip_address}"
        response = requests.get(url, timeout=5)
        data = response.json()
        
        if data['status'] == 'success':
            return {
                'IP': data.get('query'),
                '国家': data.get('country'),
                '地区': data.get('regionName'),
                '城市': data.get('city'),
                'ISP': data.get('isp'),
                '经度': data.get('lon'),
                '纬度': data.get('lat')
            }
        else:
            return {'错误': '查询失败'}
    except Exception as e:
        return {'错误': str(e)}

# 示例
test_ips = ['8.8.8.8', '1.1.1.1']

for ip in test_ips:
    print(f"\n查询 IP: {ip}")
    location = get_ip_location(ip)
    for key, value in location.items():
        print(f"  {key}: {value}")


查询 IP: 8.8.8.8
  IP: 8.8.8.8
  国家: United States
  地区: Virginia
  城市: Ashburn
  ISP: Google LLC
  经度: -77.5
  纬度: 39.03

查询 IP: 1.1.1.1
  IP: 1.1.1.1
  国家: Hong Kong
  地区: Central and Western District
  城市: Hong Kong
  ISP: Cloudflare, Inc
  经度: 114.1693
  纬度: 22.3193


## 挑战练习: 暴力破解模拟器

**任务**: 创建一个模拟的暴力破解工具和防护系统

**场景**: 模拟攻击者尝试破解密码，以及防御系统的响应

**要求**:
1. 实现密码尝试功能
2. 实现速率限制
3. 实现账户锁定机制
4. 记录攻击日志

In [ ]:
import time
import hashlib
from datetime import datetime, timedelta

class LoginSystem:
    """
    带有安全防护的登录系统模拟
    """
    
    def __init__(self, max_attempts=3, lockout_time=300):
        """
        参数:
            max_attempts: 最大尝试次数
            lockout_time: 锁定时间(秒)
        """
        # 存储用户密码哈希
        self.users = {
            'admin': self._hash_password('SecureP@ss123')
        }
        
        # 跟踪登录尝试
        self.attempts = {}
        self.lockout = {}
        
        self.max_attempts = max_attempts
        self.lockout_time = lockout_time
    
    def _hash_password(self, password):
        """计算密码哈希"""
        return hashlib.sha256(password.encode()).hexdigest()
    
    def login(self, username, password):
        """
        尝试登录
        
        返回:
            (success, message) 元组
        """
        # TODO: 实现登录逻辑
        # 1. 检查账户是否被锁定
        # 2. 验证用户名和密码
        # 3. 记录失败尝试
        # 4. 达到最大尝试次数后锁定账户
        
        pass
    
    def brute_force_attack(self, username, password_list):
        """
        模拟暴力破解攻击
        
        参数:
            username: 目标用户名
            password_list: 密码字典
        """
        print(f"开始暴力破解用户: {username}")
        print(f"密码字典大小: {len(password_list)}\n")
        
        for i, password in enumerate(password_list, 1):
            success, message = self.login(username, password)
            print(f"尝试 {i}: {password} - {message}")
            
            if success:
                print(f"\n破解成功! 密码是: {password}")
                return True
            
            if "锁定" in message:
                print(f"\n账户已被锁定，攻击失败")
                return False
            
            time.sleep(0.1)  # 模拟网络延迟
        
        print(f"\n破解失败: 尝试了所有密码")
        return False

# 测试
system = LoginSystem(max_attempts=3, lockout_time=10)

# 常见弱密码字典
weak_passwords = [
    'password', '123456', 'admin', 'letmein', 
    'qwerty', 'SecureP@ss123', 'password123'
]

# system.brute_force_attack('admin', weak_passwords)